# CodeBERT

In [1]:
!nvidia-smi

Mon Dec  6 19:16:24 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 470.57.02    CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           On   | 00000001:00:00.0 Off |                    0 |
| N/A   33C    P8    34W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [2]:
import torch

"cuda" if torch.cuda.is_available() else "cpu"

'cuda'

In [3]:
!pip -q install transformers

## Mount GDrive

In [4]:
DRIVE_PATH = '/mnt/batch/tasks/shared/LS_root/mounts/clusters/teslak80code/code/Users/GUSTAVO.PEREIRA87'
DATA_PATH = DRIVE_PATH + '/codesummarization_data'

In [8]:
DATA_PATH

'/mnt/batch/tasks/shared/LS_root/mounts/clusters/teslak80code/code/Users/GUSTAVO.PEREIRA87/codesummarization_data'

## Clone CodeBert repo

In [4]:
!git clone --depth 1 https://github.com/guoday/CodeBERT.git

Cloning into 'CodeBERT'...
remote: Enumerating objects: 51, done.
remote: Counting objects: 100% (51/51), done.
remote: Compressing objects: 100% (49/49), done.
remote: Total 51 (delta 7), reused 35 (delta 2), pack-reused 0
Unpacking objects: 100% (51/51), done.


## Fine-Tune

In [9]:
%%sh

cd cod_CodeBert/CodeBERT/CodeBERT/code2nl

lang="java" #programming language
lr=5e-5
batch_size=16
beam_size=5
source_length=256
target_length=128
data_dir=/mnt/batch/tasks/shared/LS_root/mounts/clusters/teslak80code/code/Users/GUSTAVO.PEREIRA87
output_dir=$data_dir/model
train_file=$data_dir/codesummarization_data/train/train.jsonl
dev_file=$data_dir/codesummarization_data/dev/dev.jsonl
eval_steps=500 #400 for ruby, 600 for javascript, 1000 for others
train_steps=10000 #20000 for ruby, 30000 for javascript, 50000 for others
pretrained_model=microsoft/codebert-base #Roberta: roberta-base

python run.py --do_train --do_eval \
--model_type roberta \
--model_name_or_path $pretrained_model \
--train_filename $train_file \
--dev_filename $dev_file \
--output_dir $output_dir \
--max_source_length $source_length \
--max_target_length $target_length \
--beam_size $beam_size \
--train_batch_size $batch_size \
--eval_batch_size $batch_size \
--learning_rate $lr \
--train_steps $train_steps \
--eval_steps $eval_steps | tee -a train_test_metrics.txt

Process is interrupted.


***** Running evaluation *****
12/08/2021 03:15:00 - INFO - __main__ -     Num examples = 8714
12/08/2021 03:15:00 - INFO - __main__ -     Batch size = 16
12/08/2021 03:21:54 - INFO - __main__ -     eval_ppl = 26.37854
12/08/2021 03:21:54 - INFO - __main__ -     global_step = 30000
12/08/2021 03:21:54 - INFO - __main__ -     train_loss = 3.056
12/08/2021 03:21:54 - INFO - __main__ -     ********************
12/08/2021 03:21:59 - INFO - __main__ -     Best ppl:26.37854
12/08/2021 03:21:59 - INFO - __main__ -     ********************
Total: 1000
12/08/2021 03:31:42 - INFO - __main__ -     bleu-4 = 15.29 
12/08/2021 03:31:42 - INFO - __main__ -     ********************
12/08/2021 03:31:42 - INFO - __main__ -     Best bleu:15.29
12/08/2021 03:31:42 - INFO - __main__ -     ********************
loss 2.9972: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 30000/30000 [26:57:10<00:00,  3.23s/it]


# Inference and Evaluation

In [5]:
%%sh

cd cod_CodeBert/CodeBERT/CodeBERT/code2nl

lang="java" #programming language
beam_size=5
batch_size=16
source_length=256
target_length=128
data_dir=/mnt/batch/tasks/shared/LS_root/mounts/clusters/teslak80code/code/Users/GUSTAVO.PEREIRA87
output_dir=$data_dir/model
dev_file=$data_dir/codesummarization_data/dev/dev.jsonl
test_file=$data_dir/codesummarization_data/test/test.jsonl
test_model=$output_dir/checkpoint-best-bleu/pytorch_model.bin #checkpoint for test

python run.py --do_test \
--model_type roberta \
--model_name_or_path microsoft/codebert-base \
--load_model_path $test_model \
--dev_filename $dev_file \
--test_filename $test_file \
--output_dir $output_dir \
--max_source_length $source_length \
--max_target_length $target_length \
--beam_size $beam_size \
--eval_batch_size $batch_size

12/06/2021 16:43:37 - INFO - __main__ -   Namespace(adam_epsilon=1e-08, beam_size=5, config_name='', dev_filename='/content/drive/MyDrive/codesummarization_data/dev/dev.jsonl', do_eval=False, do_lower_case=False, do_test=True, do_train=False, eval_batch_size=128, eval_steps=-1, gradient_accumulation_steps=1, learning_rate=5e-05, load_model_path='/content/drive/MyDrive/model/checkpoint-best-bleu/pytorch_model.bin', local_rank=-1, max_grad_norm=1.0, max_source_length=256, max_steps=-1, max_target_length=128, model_name_or_path='microsoft/codebert-base', model_type='roberta', no_cuda=False, num_train_epochs=3.0, output_dir='/content/drive/MyDrive/model', seed=42, test_filename='/content/drive/MyDrive/codesummarization_data/test/test.jsonl', tokenizer_name='', train_batch_size=8, train_filename=None, train_steps=-1, warmup_steps=0, weight_decay=0.0)
12/06/2021 16:43:37 - WARNING - __main__ -   Process rank: -1, device: cuda, n_gpu: 1, distributed training: False
Downloading: 100% 498/498 [

CalledProcessError: ignored

In [ ]:
# import torch
# from transformers import RobertaTokenizer, RobertaConfig, RobertaModel

# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")
# model = RobertaModel.from_pretrained("microsoft/codebert-base")
# model.to(device)